# setup

In [ ]:
run_test=True
# run_test=False
epochs=(1,1,1,0)
epochs=(15,500,3,0)
cutdate='2025-11-18'
valData=''
use_amp=True
use_amp=False
# valData='2512'
testData=''
testData='new/'
checkpoints="./checkpoints/"

weight=(0.81,1.01,0.03,0)
itr=5
metrics_cnt=6
import torch,sys

if 'google.colab' in str(get_ipython()):
  from google.colab import drive
  drive.mount('/content/drive')
  mydrive='/content/drive/MyDrive/volrt/'
  source_path=mydrive+"git/Crossformer"
  tables = [ 'volvTSLA.csv', 'volvAAPL.csv', 'volvGOOG.csv','volvNVDA.csv', 'volvMETA.csv', 'volvMSFT.csv', 'volvAMZN.csv', ]
  batch_size=int(torch.cuda.get_device_properties(0).total_memory/1e9+.5)*32
  import os,sys
  if not os.path.exists(source_path):
    %cd $mydrive/git
    !git clone https://github.com/jerronl/Crossformer.git
    %cd $source_path
  else:
    %cd $source_path
    # !git reset --hard HEAD
    # !git pull origin master
  %pip install einops #scikit-learn
  sys.path.append( source_path)
else:
  tables = [ 'volvAAPL.csv']
  testData=''
  mydrive= 'E:/mydoc/git/trade/analyics/'
  batch_size=32
  sys.path.append("G:/git/Crossformer1/")
  itr=1
  %load_ext autoreload
  %autoreload 2
%cd $mydrive


In [ ]:
from cross_exp.exp_crossformer import Exp_crossformer


from utils.tools import init_args,update_args
import seaborn as sns, numpy as np,math
import matplotlib.pyplot as plt
from data.data_def import set_cat
set_cat(20)
args=init_args()
data_parser={}


def regplot(dep_var, metrics, cols, itr=None, figsize=4):
    cnt = len(dep_var) - dep_var.count("_")
    figs = min(cnt, cols)
    _, axes = plt.subplots(math.ceil(cnt / figs), figs, figsize=(figsize*figs,figsize*max(1,(cnt-1)//figs+1)))
    j = 0

    for i, name in enumerate(dep_var):
        if name != "_":
            # if itr:
            name=f'{itr}_{name}'
            axs = axes.flat[j] if figs > 1 else axes
            j = j + 1
            left, right = 999, -999
            ax2=axs.twinx()
            max_hist=0
            for ii in range(len(results)):
                preds, trues, _ = results[ii]
                h,_,_=ax2.hist(trues[:, i], density=True,alpha=0.2,color=f"C{ii}")
                max_hist=max(max_hist,np.max(h))
                sns.regplot(
                    ax=axs,
                    x=trues[:, i],
                    y=preds[:, i],
                    scatter_kws={"color": f"C{ii}", "alpha": 0.2},
                    line_kws={"color": f"C{ii}", "alpha": 0.4},
                    label=labels[1][ii],
                )
                mask = ~np.isnan(trues[:, i])
                if not dep_var[i][:3] in ["dtm", "pmc"]:
                    left = min(left, max(np.min(trues[:, i][mask]), -5))
                    right = max(right, min(np.max(trues[:, i][mask]), 5))
                else:
                    left = min(left, np.min(trues[:, i][mask]))
                    right = max(right, np.max(trues[:, i][mask]))
            ax2.tick_params(axis='y', labelleft=False, labelright=False, left=False, right=False)
            ax2.set_ylim(0, max_hist*3)
            ax2.set_ylabel('')
            axs.set_title(name)
            xmin, xmax = axs.get_ylim()
            left, right=min(-0.01,min(left,xmin)),max(0.01,max(xmax, right))
            axs.set_xlim(left=left, right=right)
            axs.set_ylim(left, right)
            axs.legend()
    metric = []
    for ii in range(len(results)):
        _, _, m = results[ii]
        metric.append(m)

    metrics = np.append(
        metrics, np.array(metric).reshape([1, len(metric), len(m)]), axis=0
    )

    plt.show()
    return metrics



def plot_metric(metrics, labels, *args, **kwargs):
    a, b, c = metrics.shape
    _, axs = plt.subplots(
        nrows=math.ceil(c / 2),
        ncols=2, figsize=(16, 16),
    )
    for i in range(c):
        ax = (
            axs[i // 2, i % 2] if c > 1 else axs
        )  # Handle the case when c=1 to avoid indexing errors
        for j in range(a):
            ax.plot(
                metrics[j, :, i], label=labels[0][j], *args, **kwargs
            )  # Plot each series in the i-th plot
        ax.set_title(labels[2][i])
        ax.legend()  # Show legend in each subplot

        # Set custom x-axis labels
        ax.set_xticks(range(b))  # Set x-tick positions for all 'b' points
        ax.set_xticklabels(labels[1])  # Set x-tick labels

    plt.tight_layout()
    plt.show()

def plot_hlc():
    global results, labels
    dep_var = data_names(data_columns(test_set), 20)[0]
    labels = [
        [f'm{i}' for i in range(itr)],  # Model identifiers
        tables,                         # Table names
        ["mae", "mse", "rmse", "mape", "wmse", "accr"]  # Metric labels
    ]
    metrics = np.empty((0, len(tables), metrics_cnt))
    n_categories = data_columns(test_set)["ycat"]
    x = np.arange(1, n_categories + 1)
    for i in range(itr):
        setting = update_args(args, data_parser, i)
        DatasetMTS.clear()
        exp = Exp_crossformer(args)
        print(f">>>>>>> Testing: {setting} <<<<<<<<<<<<<<<<<<<<<<<<<<<")
        results = []

        for table in tables:
            results.append(exp.test(setting, 'prcs', True, data_path=[table], inverse=1))

        metrics = regplot(dep_var, metrics, 4, i)

        cols=4
        rows = (len(results) - 1) // cols + 1

        _, axes = plt.subplots(rows*2, cols, figsize=(16, cols * rows*2))
        axes = axes.flatten()

        for j, result in enumerate(results):
            preds, trues, _ = result
            preds = preds[:, -n_categories:]
            trues = np.round(trues[:, 0]).astype(int)
            df = pd.DataFrame(preds, columns=[str(i) for i in range(1, n_categories + 1)])
            df['True_Label'] = trues

            summed_probs = df.groupby('True_Label').mean().T
            xi=[float(x) + 0.5 for x in summed_probs.index]
            half=max(summed_probs.columns)/2
            third=max(summed_probs.columns)//3+1
            ls=['--','-',':']
            for k in (0,1):
              ci=0
              ax = axes[int(j//cols)*cols*2+j%cols+k*cols]
              ax.hist(trues, bins=np.arange(0.5, n_categories + 0.5),alpha=0.2, density=True)
              for m in range(k,len(summed_probs.columns),2):
                  col=summed_probs.columns[m]
                  ax.plot(xi, summed_probs[col],
                          label=f'Col {col}',
                          linestyle=ls[col // third ],
                          color=f'C{ci % 10}',alpha=0.5)
                  ci+=1

                  ax.text(col-.5, summed_probs[col][col-1], f'{col}',
                          fontsize=8, verticalalignment='center', horizontalalignment='left')
                  ax.legend(loc='center left', bbox_to_anchor=(1.05, 0.5))

              ax.set_title(f'{tables[j]}_{i}')
              ax.set_ylim(0.0, 0.2)
              ax.set_xlim(0.5, n_categories + 0.5)
              ax.set_xticks(x)
              ax.set_xticklabels([str(i) for i in x], rotation=-45)

        plt.tight_layout()
        plt.show()

    print(metrics)
    plot_metric(metrics, labels)


# train

In [ ]:
# ===========================
#   FINAL TRAINING SCRIPT
# ===========================

import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from copy import deepcopy

from cross_exp.exp_crossformer import Exp_crossformer
from data.data_loader import DatasetMTS

# -----------------------------------------
# Heatmap for residual (auto-truncate logits)
# -----------------------------------------
def plot_residual_heatmap(preds, trues, title, save_path, bins=50, show_inline=True):
    """
    画预测 vs 真实的 2D residual heatmap
    - vols: preds/trues 维度一致
    - prcs: preds 多出 ycat logits，自动截掉
    """
    import numpy as np
    import matplotlib.pyplot as plt
    import os

    preds = np.asarray(preds)
    trues = np.asarray(trues)

    # prcs: preds 多出分类 logits，自动截掉
    if preds.ndim == 2 and trues.ndim == 2 and preds.shape[1] > trues.shape[1]:
        preds = preds[:, :trues.shape[1]]

    preds_flat = preds.reshape(-1)
    trues_flat = trues.reshape(-1)

    # 对齐长度（极端情况下）
    if preds_flat.size != trues_flat.size:
        L = min(preds_flat.size, trues_flat.size)
        preds_flat = preds_flat[:L]
        trues_flat = trues_flat[:L]

    # 去 NaN
    mask = ~np.isnan(preds_flat) & ~np.isnan(trues_flat)
    preds_flat = preds_flat[mask]
    trues_flat = trues_flat[mask]

    plt.figure(figsize=(5, 4))
    plt.hist2d(trues_flat, preds_flat, bins=bins)
    plt.colorbar(label="count")
    plt.xlabel("True")
    plt.ylabel("Predicted")
    plt.title(title)
    plt.tight_layout()

    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    plt.savefig(save_path, dpi=150)

    if show_inline:
        plt.show()
    else:
        plt.close()


# ------------------------
#  Prepare directories
# ------------------------
SUMMARY_DIR = os.path.join(checkpoints, "summaries")
FIG_DIR = os.path.join(checkpoints, "figures")
os.makedirs(SUMMARY_DIR, exist_ok=True)
os.makedirs(FIG_DIR, exist_ok=True)
SUMMARY_PATH = os.path.join(SUMMARY_DIR, "summary.csv")

# ===========================
#       PARAMETER GRID
# ===========================

BASE_LR = 0.01
RUN_PRCS = True  # 单独跑 prcs

# 只 sweep vols 的参数
VOL_GRID = []
lambda_vars = [0.0005, 0.001, 0.002]
weight_overs = [1.5, 2.0, 3.0]

for lv in lambda_vars:
    for wo in weight_overs:
        name = f"LV{lv}_WO{wo}"
        VOL_GRID.append((name, lv, 2.0, 0.0, wo))  
        # (name, lambda_var, lambda_ce, lambda_shape, weight_over)

# ------------------------
# Base data parser
# ------------------------
base_data_parser = {
    "vols": {
        "data": "vols",
        "patience": epochs[0],
        "train_epochs": epochs[1],
        "learning_rate": BASE_LR,
        "data_split": [0.7, 0.15, 0.15],
        "batch_size": batch_size * 2 // 5,
        "e_layers": 5,
        "d_model": 512,
        "lradj": "type2",
        "checkpoints": checkpoints,
        "root_path": mydrive + valData,
        "data_path": tables,
        "profile_mode": False,
        "use_amp": use_amp,
    },
    "prcs": {
        "data": "prcs",
        "weight": 0.0,        # prcs 固定 λ_var=0
        "over_weight": weight[3],
    },
}

# ------------------------
#  Helper: apply parser
# ------------------------
def update_args_from_parser(args, data_parser, run_name, ds):
    cfg = data_parser[ds]
    for k, v in cfg.items():
        setattr(args, k, v)
    return f"{run_name}_{ds}"

# Clear dataset ONCE (保证所有组合使用同一划分）
DatasetMTS.clear()

summary_rows = []

# ===========================
#       TRAINING LOOP
# ===========================
for name, lambda_var, lambda_ce, lambda_shape, weight_over in VOL_GRID:

    print(f"\n===========  VOLS: {name}  ===========")

    # ---- build data parser ----
    data_parser = deepcopy(base_data_parser)
    data_parser["vols"]["learning_rate"] = BASE_LR
    data_parser["vols"]["weight"] = lambda_var
    data_parser["vols"]["lambda_ce"] = lambda_ce
    data_parser["vols"]["lambda_shape"] = lambda_shape
    data_parser["vols"]["weight_over"] = weight_over

    # ---------------------
    #    train vols
    # ---------------------
    setting = update_args_from_parser(args, data_parser, name, "vols")
    exp = Exp_crossformer(args)

    print(f"Training vols: {setting}")
    exp.train(setting, "vols")

    preds, trues, metrics = exp.test(
        setting, "vols", save_pred=True, data_path=[tables[-1]], inverse=True
    )
    fig_path = os.path.join(FIG_DIR, f"{setting}_residual_vols.png")
    plot_residual_heatmap(preds, trues, f"{setting} vols", fig_path)

    if metrics is not None:
        mae, mse, rmse, mape, mdst, accr = metrics
        summary_rows.append({
            "exp": name, "dataset": "vols",
            "lambda_var": lambda_var,
            "weight_over": weight_over,
            "lambda_ce": lambda_ce,
            "lambda_shape": lambda_shape,
            "mae": mae, "mse": mse, "rmse": rmse,
            "mape": mape, "mdst": mdst, "accr": accr
        })


    # ---------------------
    #    train prcs (optional)
    # ---------------------
    if RUN_PRCS:
        print(f"\n---- PRCS for {name} ----")

        # prcs 固定 λ_var=0，只同步 ce/shape
        data_parser["prcs"]["lambda_ce"] = lambda_ce
        data_parser["prcs"]["lambda_shape"] = lambda_shape
        data_parser["prcs"]["weight_over"] = 1.0  # prcs 不用 asymmetry

        setting_p = update_args_from_parser(args, data_parser, name, "prcs")
        exp_p = Exp_crossformer(args)
        exp_p.train(setting_p, "prcs")

        preds_p, trues_p, metrics_p = exp_p.test(
            setting_p, "prcs", save_pred=False, data_path=[tables[-1]], inverse=True
        )

        fig_path_p = os.path.join(FIG_DIR, f"{setting_p}_residual_prcs.png")
        plot_residual_heatmap(preds_p, trues_p, f"{setting_p} prcs", fig_path_p)

        if metrics_p is not None:
            mae, mse, rmse, mape, mdst, accr = metrics_p
            summary_rows.append({
                "exp": name, "dataset": "prcs",
                "lambda_var": 0.0,
                "weight_over": 1.0,
                "lambda_ce": lambda_ce,
                "lambda_shape": lambda_shape,
                "mae": mae, "mse": mse, "rmse": rmse,
                "mape": mape, "mdst": mdst, "accr": accr
            })

# ===========================
#     SAVE SUMMARY
# ===========================
df = pd.DataFrame(summary_rows)
df.to_csv(SUMMARY_PATH, index=False)

print("\nSummary saved:", SUMMARY_PATH)
df


In [ ]:
if not run_test:
    from google.colab import runtime
    runtime.unassign()

In [ ]:
if not run_test:
    from cross_exp.exp_crossformer import Exp_crossformer
    from data.data_loader import DatasetMTS
    data_parser = {
        "vols": {
            'data':'vols',
            "patience":epochs[0],
            "train_epochs":epochs[1],
            'learning_rate':0.01,
            'data_split':[0.7,0.15,0.15],
            'batch_size':batch_size*2//5,
            'e_layers':5,
            'd_model':512,
            'lradj':'type2',
            "checkpoints":checkpoints,
            'root_path':mydrive+valData,
            'data_path':tables,
            'weight':weight[0],
            'over_weight':weight[2],
            'profile_mode':False,
            'use_amp':use_amp
        },
        "prcs":{
            'data':'prcs',
            'weight':weight[1],
            'over_weight':weight[3],
        }
        }
    s=epochs[3]
    for _ in range(epochs[2]):
        data_parser["vols"]["learning_rate"]=.01
        for i in range(epochs[2]):
            for ii in range(s,itr):
                # setting record of experiments
                # data_parser["vols"]['weight']=weight[0]
                setting=update_args(args,data_parser,ii)
                DatasetMTS.clear()
                # drive.mount('/content/drive/',force_remount=True)
                !google-drive-ocamlfuse -cc

                exp = Exp_crossformer(args)  # set experiments
                print(f"start training :lr={data_parser["vols"]["learning_rate"]} {setting}")
                exp.train(setting, "vols")

                print(f">>>>>>>testing : {setting}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<")
                preds, trues,_ = exp.test(setting, 'vols', True, data_path=[tables[-1]], inverse=True)
                print(preds.shape, trues.shape)

                # data_parser["vols"]['weight']=weight[1]
                setting=update_args(args,data_parser,ii,'prcs')
                DatasetMTS.clear()
                exp = Exp_crossformer(args)  # set experiments
                print(f"start training :lr={data_parser["vols"]["learning_rate"]} {setting}")
                exp.train(setting, "prcs")
                preds, trues,_ = exp.test(setting, 'prcs', True, data_path=[tables[-1]], inverse=True)
                print(preds.shape, trues.shape)
            data_parser["vols"]["learning_rate"]/=3.
            s=0

# test

In [ ]:
from data.data_loader import DatasetMTS
from data.data_def import data_columns, data_names
data_parser = {
    "vols": {
        'e_layers':5,
        'd_model':512,
        'lradj':'type2',
        'weight':weight[0],
        'root_path':mydrive+valData,
    },
    }
test_set='vols'
dep_var=data_names(data_columns(test_set),20)[0]
labels=[[f'm{i}' for i in range(itr)],tables,["mae", "mse", "rmse", "mape", "wmse", "accr"]]
metrics=np.empty((0,len(tables),metrics_cnt))
for i in range(itr):
  setting=update_args(args,data_parser,i)
  DatasetMTS.clear()
  exp = Exp_crossformer(args)
  print(f">>>>>>>testing : {setting}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<")
  results = []
  for table in tables:
      results.append(exp.test(setting, 'vols', True, data_path=[table], inverse=True))
  metrics=regplot(dep_var,metrics,4,i)

print(metrics)
plot_metric(metrics,labels)

## hlc

In [ ]:
from data.data_loader import DatasetMTS
from data.data_def import data_columns, data_names
import pandas as pd

test_set='prcs'
data_parser = {
    "vols": {
        'e_layers':5,
        'd_model':512,
        'lradj':'type2',
        'root_path':mydrive+valData,
        'weight':weight[1]#0.8#
    },
    }
plot_hlc()

## horizon


In [ ]:
from data.data_loader import DatasetMTS
from cross_exp.exp_crossformer import Exp_crossformer
test_set='vols'

labels=[[f'm{i}' for i in range(itr)] ,[f'h{5-h}' for h in range(5)],["mae", "mse", "rmse", "mape", "wmse", "accr"]]
dep_var=data_names(data_columns(test_set),20)[0]
metrics=np.empty((0,len(labels[1]),len(labels[2])))
for i in range(itr):
  results = []
  for h in range(5):
    data_parser = {
    "vols": {
        'e_layers':5,
        'd_model':512,
        'lradj':'type2',
        "query": f"floor(horizon)=={5-h}",
        'weight':weight[0],
        'root_path':mydrive+valData,
        'data_path':tables,    
    },
    }
    setting=update_args(args,data_parser,i)
    DatasetMTS.clear()
    exp = Exp_crossformer(args)
    print(f">>>>>>>testing : {data_parser['vols']['query']} m{i}h{5-h}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<")
    results.append(exp.test(setting, 'vols', True, inverse=True))
  metrics=regplot(dep_var,metrics,4,i)

print(metrics)
plot_metric(metrics,labels)

## cutline

In [ ]:
from cross_exp.exp_crossformer import Exp_crossformer
from data.data_loader import DatasetMTS
from data.data_def import data_columns, data_names
test_set='vols'

data_parser = {
    "vols": {
        'e_layers':5,
        'd_model':512,
        "query": f"date>'#{cutdate}'",
        'root_path':mydrive+testData,
        'weight':weight[0],    
    },
    }

dep_var=data_names(data_columns(test_set),20)[0]
labels=[[f'm{i}' for i in range(itr)],tables,["mae", "mse", "rmse", "mape", "wmse", "accr"]]
metrics=np.empty((0,len(tables),metrics_cnt))
for i in range(itr):
  setting=update_args(args,data_parser,i)
  DatasetMTS.clear()
  exp = Exp_crossformer(args)
  print(f">>>>>>>testing : {setting}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<")
  results = []
  for table in tables:
      results.append(exp.test(setting, 'vols', True, data_path=[table], inverse=True))
  metrics=regplot(dep_var,metrics,4,i)

print(metrics)
plot_metric(metrics,labels)

## hlc

In [ ]:
from data.data_loader import DatasetMTS
from data.data_def import data_columns, data_names
import pandas as pd

test_set='prcs'
data_parser = {
    "vols": {
        'e_layers':5,
        'd_model':512,
        'lradj':'type2',
        "query": f"date>'#{cutdate}'",
        'root_path':mydrive+testData,
        'weight':weight[1],
        },
    }
plot_hlc()

## horizon


In [ ]:
from data.data_loader import DatasetMTS
from cross_exp.exp_crossformer import Exp_crossformer
from data.data_def import data_columns, data_names
test_set='vols'
labels=[[f'm{i}' for i in range(itr)] ,[f'h{5-h}' for h in range(5)],["mae", "mse", "rmse", "mape", "wmse", "accr"]]
dep_var=data_names(data_columns(test_set),20)[0]
metrics=np.empty((0,len(labels[1]),len(labels[2])))
for i in range(itr):
  results = []
  for h in range(5):
    data_parser = {
    "vols": {
        'e_layers':5,
        'd_model':512,
        'lradj':'type2',
        'root_path':mydrive+testData,
        "query": f"date>'#{cutdate}' and floor(horizon)=={5-h}",
        'weight':weight[0],
        'data_path':tables,    
    },
    }
    setting=update_args(args,data_parser,i)
    DatasetMTS.clear()
    exp = Exp_crossformer(args)
    print(f">>>>>>>testing : {data_parser['vols']['query']} m{i}h{5-h}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<")
    results.append(exp.test(setting, 'vols', True, inverse=True))
  metrics=regplot(dep_var,metrics,4,i)

print(metrics)
plot_metric(metrics,labels)

# end

In [ ]:
plt.show()
from google.colab import runtime
runtime.unassign()